In [1]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

import time
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any
from pathlib import Path
from IPython.display import clear_output
from collections import defaultdict

import os
import dotenv
from qubx import QubxLogConfig
from qubx.utils.runner import run_ccxt_trading

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import TriggerEvent, Trade, MarketEvent, Instrument, SubscriptionType
from qubx.core.interfaces import IStrategyContext, IStrategy
from qubx.connectors.ccxt.ccxt_connector import CCXTExchangesConnector
from qubx.connectors.ccxt.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config
from qubx.pandaz import scols
from qubx.backtester.simulator import SimulatedTrading
from qubx.utils.runner import run_ccxt_paper_trading
from qubx.utils.collections import TimeLimitedDeque


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. 0.4.0
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


## 1.0 Live paper trading

In [ ]:
class TradeTestStrat(IStrategy):
    _data_counter: int = 0
    _data_to_buffer: dict[tuple[str, Instrument], TimeLimitedDeque]

    def on_init(self, ctx: IStrategyContext):
        # ctx.set_base_subscription(SubscriptionType.ORDERBOOK)
        # ctx.set_base_subscription(SubscriptionType.OHLC)
        # ctx.set_warmup(SubscriptionType.OHLC, "1h")
        ctx.set_base_subscription(SubscriptionType.TRADE)
        self._data_to_buffer = defaultdict(lambda: TimeLimitedDeque("1Min", lambda x: x.time, unit="ns"))

    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        self._data_to_buffer[(data.type, data.instrument)].append(data.data)
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)

    def get_data(self, type: str, instrument: Instrument) -> list:
        return list(self._data_to_buffer[(type, instrument)])


ctx = run_ccxt_paper_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE.UM",
    symbols=["BTCUSDT", "ETHUSDT"],
    # symbols=["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT"],
    blocking=False,
)

In [ ]:
ctx.stop()

In [ ]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

In [ ]:
len(ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE))

In [ ]:
i1 = ctx.instruments[1]
obs = stg.get_data("ohlc", i1)
print(f"Instrument: {i1}")
for i in range(1, 5):
    print(obs[-i])

### Add trade subscription and remove it

In [ ]:
ctx.subscribe(ctx.instruments, SubscriptionType.TRADE)

In [ ]:
trades = stg.get_data("trade", ctx.instruments[0])
trades[-5:]

In [ ]:
stg.ctx.unsubscribe(ctx.instruments, SubscriptionType.TRADE)

### Add new instrument to the universe

In [ ]:
s1 = lookup.find_symbol("BINANCE.UM", "XRPUSDT"); assert s1 is not None
ctx.set_universe(list(set(ctx.instruments) | {s1}))

In [ ]:
new_instruments = ctx.instruments[-2:]
print(new_instruments)
ctx.set_universe(new_instruments)

In [ ]:
obs = stg.get_data("orderbook", s1)
obs[-1]

In [ ]:
s2 = lookup.find_symbol("BINANCE.UM", "ADAUSDT"); assert s2 is not None
stg.ctx.set_universe(list(set(stg.ctx.instruments) | {s2}))

In [ ]:
trades = stg.get_data("trade", s2)
trades[-5:]

In [ ]:
ctx.stop()

## 2.0 Live execution

In [ ]:
QubxLogConfig.set_log_level("DEBUG")


class TradeTestStrat(IStrategy):
    _data_counter: int = 0

    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(SubscriptionType.OHLC, timeframe="1m")
        ctx.set_warmup(SubscriptionType.OHLC, "1h")
    
    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)


dotenv.load_dotenv(
    "/mnt/HC_Volume_100695026/home/shared/devs/Qubx/debug/.env.binance.yuriy"
)

credentials = {
    "apiKey": os.getenv("BINANCE_KEY"),
    "secret": os.getenv("BINANCE_SECRET")
}

ctx = run_ccxt_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE",
    symbols=["BTCUSDT", "ETHUSDT"],
    credentials=credentials,
    blocking=False,
)

2024-11-15 09:41:25.487 [ ℹ️ ] BINANCE loading ...
2024-11-15 09:41:28.763 [ 🐞 ] NTP offset controller thread is started
2024-11-15 09:41:28.790 [ ℹ️ ] binance initialized - current time 2024-11-15T09:41:28.774249728
2024-11-15 09:41:28.790 [ ℹ️ ] Loading account data for Binance
2024-11-15 09:41:29.517 [ ⚠️ ] qubx.connectors.ccxt.ccxt_utils:ccxt_restore_position_from_deals:102 - Couldn't restore full deals history for BTCUSDT symbol. Qubx will use zero position !
2024-11-15 09:41:30.007 [ ⚠️ ] qubx.connectors.ccxt.ccxt_utils:ccxt_restore_position_from_deals:102 - Couldn't restore full deals history for ETHUSDT symbol. Qubx will use zero position !
2024-11-15 09:41:30.010 [ 🐞 ] Running OHLC warmup for {BINANCE:CRYPTO:BTCUSDT, BINANCE:CRYPTO:ETHUSDT} with period 1h
2024-11-15 09:41:30.010 [ 🐞 ] Listening to BTCUSDT executions
2024-11-15 09:41:30.011 [ 🐞 ] Listening to ETHUSDT executions
2024-11-15 09:41:30.012 [ ℹ️ ] (StrategyContext) Start processing market data
2024-11-15 09:41:30.013

2024-11-15 09:41:32.280 [ ℹ️ ] BINANCE:CRYPTO:BTCUSDT: loaded 60 1m bars
2024-11-15 09:41:32.281 [ 🐞 ] Invoking TradeTestStrat on_fit
2024-11-15 09:41:32.282 [ 🐞 ] TradeTestStrat is fitted
2024-11-15 09:41:32.312 [ ℹ️ ] BINANCE:CRYPTO:ETHUSDT: loaded 60 1m bars
2024-11-15 09:41:32.313 [ 🐞 ] Listening to BTCUSDT,ETHUSDT ohlc (warmup_period=1h)
2024-11-15 09:41:36.462 [ 🐞 ] New instruments for trade warmup: {BINANCE:CRYPTO:BTCUSDT, BINANCE:CRYPTO:ETHUSDT}
2024-11-15 09:41:36.729 [ 🐞 ] Loaded 500 trades for BINANCE:CRYPTO:BTCUSDT
2024-11-15 09:41:36.749 [ 🐞 ] Loaded 500 trades for BINANCE:CRYPTO:ETHUSDT
2024-11-15 09:41:36.749 [ 🐞 ] Listening to BTCUSDT,ETHUSDT trade (warmup_period=1m)
2024-11-15 09:41:43.701 [ 🐞 ] Processed 1000 data points
2024-11-15 09:41:45.650 [ 🐞 ] Processed 2000 data points
2024-11-15 09:41:49.860 [ 🐞 ] Processed 3000 data points
2024-11-15 09:41:50.080 [ 🐞 ] Canceling existing trade subscription for {BINANCE:CRYPTO:BTCUSDT, BINANCE:CRYPTO:ETHUSDT}
2024-11-15 09:42

In [24]:
ctx.get_total_capital()

17.05809094

In [26]:
ctx.ohlc(ctx.instruments[0]).pd().tail()

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2024-11-15 09:34:00,89102.38,89179.79,89076.00,89179.79,2.643599e+06,1.058410e+06
2024-11-15 09:35:00,89179.79,89350.00,89173.29,89332.25,4.888036e+06,2.760296e+06
2024-11-15 09:36:00,89332.50,89336.39,89160.00,89227.30,2.649142e+06,1.016469e+06
2024-11-15 09:37:00,89227.29,89320.00,89227.29,89243.82,1.845508e+06,8.808934e+05
2024-11-15 09:38:00,89243.83,89312.00,89240.00,89240.00,1.316401e+06,3.701344e+05


In [15]:
o = ctx.trade(ctx.instruments[0], amount=0.00019, price=88000)
o

Order(id='32397996623', type='LIMIT', instrument=BINANCE:CRYPTO:BTCUSDT, time=Timestamp('2024-11-13 18:29:01.332000'), quantity=0.00019, price=88000.0, side='BUY', status='NEW', time_in_force='GTC', client_id='TradeTestStrat_BTCUSDT_17315225413', cost=0.0, options={})

In [16]:
ctx.cancel_order(o.id)

In [6]:
ctx.subscribe(ctx.instruments, SubscriptionType.TRADE)

In [7]:
ctx.unsubscribe(ctx.instruments, SubscriptionType.TRADE)

In [ ]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

In [ ]:
ctx.set_universe([ctx.instruments[0]])

In [18]:
ctx.subscribe(ctx.instruments, SubscriptionType.ORDERBOOK)

In [ ]:
ctx.unsubscribe(ctx.instruments, SubscriptionType.ORDERBOOK)

In [22]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

[[2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180607,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180608,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180609,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180610,
 [2024-11-13T18:30:34.078000000]	92689.37000 (0.00) take 4066180611]

In [23]:
ctx.get_data(ctx.instruments[0], SubscriptionType.ORDERBOOK)[-5:]

[[2024-11-13T18:30:43.213000000] 92687.4 (5.00577) | 92687.41 (2.88649),
 [2024-11-13T18:30:43.313000000] 92687.4 (5.36551) | 92687.41 (0.74832),
 [2024-11-13T18:30:43.413000000] 92687.4 (4.57564) | 92687.41 (0.41051),
 [2024-11-13T18:30:43.513000000] 92687.4 (4.58619) | 92687.41 (0.55437),
 [2024-11-13T18:30:43.613000000] 92687.4 (4.58658) | 92687.41 (0.55437)]

In [25]:
i1 = ctx.instruments[0]
q = ctx.quote(i1)
q

[2024-11-13T18:31:11.513000000]	92536.88000 (1.6) | 92536.89000 (2.6)

In [8]:
ctx.stop()

In [ ]:
pd.Timedelta("1h").seconds

## Tmp stuff

In [2]:
class TradeTestStrat(IStrategy):
    _data_counter: int = 0

    def on_init(self, ctx: IStrategyContext):
        ctx.set_base_subscription(SubscriptionType.OHLC, timeframe="1m")
        ctx.set_warmup(SubscriptionType.OHLC, "1h")
    
    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)


options = dotenv.dotenv_values("/mnt/HC_Volume_100695026/home/shared/devs/Qubx/.env.integration")

test_credentials = {
    "BINANCE": {
        "apiKey": options["BINANCE_SPOT_API_KEY"],
        "secret": options["BINANCE_SPOT_SECRET"]
    },
    "BINANCE.UM": {
        "apiKey": options["BINANCE_FUTURES_API_KEY"],
        "secret": options["BINANCE_FUTURES_SECRET"]
    }
}

exchange = "BINANCE.UM"

ctx = run_ccxt_trading(
    strategy=(stg := TradeTestStrat()),
    exchange=exchange,
    symbols=["BTCUSDT"],
    credentials=test_credentials[exchange],
    blocking=False,
    use_testnet=True,
    commissions="vip0_usdt",
)

i1 = ctx.instruments[0]

2024-11-17 12:49:52.256 [ ℹ️ ] (ccxt_trading) BINANCEUSDM loading ...
2024-11-17 12:49:52.858 [ 🐞 ] (ntp) NTP offset controller thread is started
2024-11-17 12:49:52.888 [ ℹ️ ] (ccxt_connector) binance.um initialized - current time 2024-11-17T12:49:52.867604992
2024-11-17 12:49:52.889 [ ℹ️ ] (ccxt_trading) Loading account data for Binance USDⓈ-M
2024-11-17 12:49:54.099 [ 🐞 ] (ccxt_connector) Running OHLC warmup for {BINANCE.UM:CRYPTO:BTCUSDT} with period 1h
2024-11-17 12:49:54.100 [ 🐞 ] (ccxt_connector) Listening to BTCUSDT executions
2024-11-17 12:49:54.100 [ ℹ️ ] (ccxt_connector) Channel BTCUSDT executions is set to True
2024-11-17 12:49:54.101 [ ℹ️ ] (context) (StrategyContext) Start processing market data
2024-11-17 12:49:54.101 [ ℹ️ ] (context) (StrategyContext) strategy is started in thread
2024-11-17 12:49:55.012 [ ℹ️ ] (ccxt_connector) BINANCE.UM:CRYPTO:BTCUSDT: loaded 60 1m bars
2024-11-17 12:49:55.013 [ 🐞 ] (ccxt_connector) Listening to BTCUSDT ohlc (warmup_period=1h)
2024-11

In [9]:
ctx.stop()

In [3]:
ctx.ohlc(i1).pd().tail()

,open,high,low,close,volume,bought_volume
timestamp,,,,,,
2024-11-17 12:45:00,91999.9,92200.0,91775.9,91920.0,693055.5611,687547.2351
2024-11-17 12:46:00,92199.9,92200.0,91775.8,92200.0,330316.9185,224223.8300
2024-11-17 12:47:00,91920.0,92199.9,91775.8,91980.0,221621.4388,11892.7275
2024-11-17 12:48:00,92039.9,92199.9,91700.0,91700.0,560996.7881,12532.8358
2024-11-17 12:49:00,91700.0,91700.0,91700.0,91700.0,13204.8000,13204.8000


In [8]:
pos = ctx.positions[i1]
pos

2024-11-17 12:50:32 [BINANCE.UM:CRYPTO:BTCUSDT] qty=0.0 entryPrice=0.0000 price=91859.9000 pnl=-0.12 value=0.00

In [7]:
ctx.trade(i1, amount=-0.002)

Order(id='4066953665', type='MARKET', instrument=BINANCE.UM:CRYPTO:BTCUSDT, time=Timestamp('2024-11-17 12:50:27.490000'), quantity=0.002, price=91800.1, side='SELL', status='CLOSED', time_in_force='GTC', client_id='TradeTestStrat_BTCUSDT_17318478133', cost=183.6002, options={})

In [12]:
trading_service = ctx._StrategyContext__broker.get_trading_service()

In [13]:
trading_service.exchange_id

'binance.um'

In [ ]:
trading_service.sync.fetch_positions()

In [9]:
ctx.stop()

In [14]:
from pprint import pprint

trading_service = ctx._StrategyContext__broker.get_trading_service()

pos_info = trading_service.sync.fetch_positions()
pprint(pos_info)

[{'collateral': 10696.94465186,
  'contractSize': 1.0,
  'contracts': 1.0,
  'datetime': '2024-11-15T14:30:49.311Z',
  'entryPrice': 90300.4398,
  'hedged': False,
  'id': None,
  'info': {'adl': '0',
           'askNotional': '0',
           'bidNotional': '0',
           'breakEvenPrice': '90264.31962408',
           'entryPrice': '90300.4398',
           'initialMargin': '4464.16366421',
           'isolatedMargin': '0',
           'isolatedWallet': '0',
           'liquidationPrice': '100494.90990235',
           'maintMargin': '396.41636642',
           'marginAsset': 'USDT',
           'markPrice': '89283.27328425',
           'notional': '-89283.27328425',
           'openOrderInitialMargin': '0',
           'positionAmt': '-1.000',
           'positionInitialMargin': '4464.16366421',
           'positionSide': 'BOTH',
           'symbol': 'BTCUSDT',
           'unRealizedProfit': '1017.16651575',
           'updateTime': '1731681049311'},
  'initialMargin': 4464.16366421,
  'in

In [29]:
pd.Timestamp(1731672816501, unit="ms").asm8

numpy.datetime64('2024-11-15T12:13:36.501')

In [ ]:
from qubx.core.basics import Position


def ccxt_construct_position(pos_infos: dict, exchange: str) -> list[Position]:
    positions = []
    for info in pos_infos:
        symbol = info["info"]["symbol"]
        instr = lookup.find_symbol(exchange, symbol)
        if instr is None:
            logger.warning(f"Could not find symbol {symbol}, skipping position...")
            continue
        pos = Position(
            instrument=instr,
            quantity=info["contracts"],
            pos_average_price=info["entryPrice"],
        )
        pos.update_market_price(pd.Timestamp(info["timestamp"], unit="ms").asm8, info["markPrice"], 1)
        positions.append(pos)
    return positions


ccxt_construct_position(pos_info, "BINANCE.UM")[0].position_avg_price

90000.0

In [ ]:
from qubx.connectors.ccxt.ccxt_customizations import BinanceQV


binance = BinanceQV(credentials)

In [ ]:
import asyncio

ohlcv = None

async def example():
    global ohlcv
    subscriptions = [["BTCUSDT", "1m"]]
    try:
        while True:
            ohlcv = await binance.watch_ohlcv_for_symbols(subscriptions)
            print(pd.Timestamp(list(list(ohlcv.values())[0].values())[0][0][0], unit='ms'))
            print(ohlcv)
    except asyncio.CancelledError:
        print("Keyboard interrupt received, exiting...")

await example()